In [ ]:
from pathlib import Path
import numpy as np
from utils import *
import wave

In [ ]:
data_root_path = Path('all_training_data')
annotations_paths = sorted(data_root_path.rglob('*.csv'))
annotations_names = [path.stem for path in annotations_paths]

In [ ]:
all_recordings_paths = sorted(Path(r'bioacoustics\experiment_2021\raw_plant_data\Aufnahmen').rglob('*.wav'))
all_recordings_paths = [p for p in all_recordings_paths if (p.stem.lower().find('test') == -1)]

In [ ]:
for p in all_recordings_paths:
    name = p.stem.lower()
    name = name.replace('uhr', '')
    name = name.replace('ohne', '')
    name = name.replace('zählung', '')
    name = name.replace('mikrofon', '')
    name = name.replace('aufnahme direkt an der blüte ', '')
    name = name.replace('aufnahme an kreis mittelpunkt ', '')
    name = name.replace(' zu- und abflug', '')
    name = name.replace('30min mik', '')
    name = name.replace(' pie', '')
    name = name.replace(',', '')
    name = name.replace('..', '.')
    name = name.replace('   ', '').strip()
    name_parts = sorted(name.split(' '), key=lambda x: x.count('.'), reverse=True)
    if len(name_parts[-1]) == 7:
        name_parts = name_parts[:-1] + [name_parts[-1][:5], name_parts[-1][5:]]
    if len(name_parts) < 3:
        name_parts = name_parts[:1] + [''] + name_parts[1:]
    if len(name_parts[0]) == 8:
        name_parts[0] = name_parts[0][:-2] + '20' + name_parts[0][-2:]
    name = '_'.join(name_parts).replace('.', '')

    if not name in annotations_names:
        continue
    save_path = annotations_paths[annotations_names.index(name)].with_suffix('.wav')
    if save_path.exists():
        continue

    audio, sr = preprocessing(p)
    frame_length = sr // 2
    audio = segmentation(audio, sr, frame_length, frame_length)
    with wave.open(str(save_path), 'wb') as f:
        f.setnchannels(1)
        f.setsampwidth(2)
        f.setframerate(sr)
        for audio_seg in audio:
            audio_seg = (audio_seg * 32767).astype(np.int16)
            f.writeframes(audio_seg.tobytes())


In [ ]:
valid_labels = ['Bee', 'NoisyBee']
correct_labels_dict = {
    'NosiyBee': 'NoisyBee',
    'Noisy': 'NoisyBee',
    'Noisy\'Bee': 'NoisyBee',
    'NoisyBe': 'NoisyBee',
    'noisyBee': 'NoisyBee',
    'bee': 'Bee',
    'NoiyBee': 'NoisyBee',
    'NoisyBeee': 'NoisyBee',
    'NoiosyBee': 'NoisyBee',
    'Noisybee': 'NoisyBee',
    'NoiysBee': 'NoisyBee',
    'nousyBee': 'NoisyBee',
    'NoioyBee': 'NoisyBee',
    'NosiyBeee': 'NoisyBee',
}

for annotations_path in annotations_paths:
    annotations = pd.read_csv(annotations_path)
    annotations['label'] = annotations['label'].apply(lambda x: x.strip() if type(x) == str else x)
    annotations['label'] = annotations['label'].apply(lambda x: correct_labels_dict.get(x, x))
    annotations.to_csv(annotations_path, index=False)
    raw_labels = annotations['label'].unique()
    if all([label in valid_labels for label in raw_labels]): continue
        
    print(annotations['label'].unique(), annotations_path)